# Importação das bibliotecas

In [51]:
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.express as px
from scipy.stats import norm

In [2]:
def obter_resumo(tipo_imovel: str, bairro: str = None, banheiro: int = None, quartos: int = None):
    query = f'  tipo_imovel == "{tipo_imovel}"'
    if bairro is not None:
        query += f' and bairro_teste == "{bairro}"  '
    if banheiro is not None:
        query += f' and banheiro == {banheiro} '
    if quartos is not None:
        query += f' and quarto == {quartos} '
    base_original = pd.read_parquet('../data/processed/base_casa_v2.parquet')
    base_original = base_original.query(query)
    return base_original

In [3]:
def obter_estatistica(base_original : pd.DataFrame):
    print('Média preço: ', round(base_original['preco'].mean(), 2))
    print('Mediana Preço:',  base_original['preco'].median())
    print('Moda: ', base_original['preco'].mode()[0])

# Preço 

- Preço Geral

In [17]:
base_original = obter_resumo(tipo_imovel='Apartamento')
base_original

,tipo_imovel,nome,preco,metragem,quarto,banheiro,garagem,ID_CASA,bairro_teste
0,Apartamento,"Apartamento com 2 Quartos à Venda, 65m²",310000.0,65,2,2,1,2699716579,Jardim Botânico
2,Apartamento,"Apartamento com 2 Quartos à Venda, 71m²",403500.0,71,2,2,2,2690493538,Bonfim Paulista
3,Apartamento,"Apartamento com 2 Quartos à Venda, 54m²",290000.0,54,2,2,1,2667652105,Nova Aliança
4,Apartamento,"Apartamento com 2 Quartos à Venda, 45m²",166420.0,45,2,1,1,2682840782,Conjunto Habitacional Jardim Das Palmeiras
5,Apartamento,"Apartamento com 2 Quartos à Venda, 64m²",370000.0,64,2,2,2,2688027260,Vila Ana Maria
...,...,...,...,...,...,...,...,...,...
9967,Apartamento,"Apartamento com 3 Quartos à Venda, 81m²",540000.0,81,3,2,2,2696267182,Vila Do Golf
9968,Apartamento,"Apartamento com Quarto à Venda, 61m²",280000.0,61,1,1,1,2476716299,Centro
9969,Apartamento,"Apartamento com 3 Quartos à Venda, 83m²",375000.0,83,3,3,2,2693112449,Jardim São Luiz
9970,Apartamento,"Apartamento com 2 Quartos à Venda, 48m²",175000.0,48,2,1,1,2572455642,Lagoinha


In [18]:
base_original['bairro_teste'].unique()

<StringArray>
[                           'Jardim Botânico',
                            'Bonfim Paulista',
                               'Nova Aliança',
 'Conjunto Habitacional Jardim Das Palmeiras',
                             'Vila Ana Maria',
                           'Jardim Republica',
                                   'Lagoinha',
                              'Jardim Sumare',
                    'Parque Dos Bandeirantes',
                                   'Ipiranga',
 ...
                                 'Royal Park',
                        'Reserva San Gabriel',
                   'Jardim Roberto Benedetti',
                            'Jardim Mosteiro',
                             'Jardim Juliana',
                            'Reserva Do Golf',
                'Jardim Valentina Figueiredo',
                          'Planalto Verde Vi',
                                   'Zona Sul',
                    'Condominio Garden Ville']
Length: 159, dtype: string

In [19]:
n = base_original.shape[0]
k = 1 + (10 / 3) * np.log10(n)
k = int(k.round(0))

k

14

In [20]:
base_original.describe()

,preco,metragem,quarto,banheiro,garagem,ID_CASA
count,8.395000e+03,8395.000000,8395.000000,8395.000000,8395.000000,8.395000e+03
mean,4.506409e+05,87.229780,2.315307,2.095771,1.463133,2.635153e+09
std,4.534368e+05,87.683108,0.766401,1.319427,2.232691,9.835162e+07
min,6.104500e+04,10.000000,1.000000,1.000000,0.000000,5.447240e+07
25%,2.150000e+05,49.000000,2.000000,1.000000,1.000000,2.603213e+09
50%,3.200000e+05,70.000000,2.000000,2.000000,1.000000,2.658819e+09
75%,5.200000e+05,103.000000,3.000000,3.000000,2.000000,2.678281e+09
max,8.000000e+06,4687.000000,20.000000,20.000000,192.000000,2.700625e+09


In [21]:
base_original['metragem_bins'] = pd.qcut(
    base_original['metragem'], q=k).astype('str')

In [22]:
base_original

,tipo_imovel,nome,preco,metragem,quarto,banheiro,garagem,ID_CASA,bairro_teste,metragem_bins
0,Apartamento,"Apartamento com 2 Quartos à Venda, 65m²",310000.0,65,2,2,1,2699716579,Jardim Botânico,"(58.0, 65.0]"
2,Apartamento,"Apartamento com 2 Quartos à Venda, 71m²",403500.0,71,2,2,2,2690493538,Bonfim Paulista,"(70.0, 77.0]"
3,Apartamento,"Apartamento com 2 Quartos à Venda, 54m²",290000.0,54,2,2,1,2667652105,Nova Aliança,"(52.0, 58.0]"
4,Apartamento,"Apartamento com 2 Quartos à Venda, 45m²",166420.0,45,2,1,1,2682840782,Conjunto Habitacional Jardim Das Palmeiras,"(42.0, 45.0]"
5,Apartamento,"Apartamento com 2 Quartos à Venda, 64m²",370000.0,64,2,2,2,2688027260,Vila Ana Maria,"(58.0, 65.0]"
...,...,...,...,...,...,...,...,...,...,...
9967,Apartamento,"Apartamento com 3 Quartos à Venda, 81m²",540000.0,81,3,2,2,2696267182,Vila Do Golf,"(77.0, 86.0]"
9968,Apartamento,"Apartamento com Quarto à Venda, 61m²",280000.0,61,1,1,1,2476716299,Centro,"(58.0, 65.0]"
9969,Apartamento,"Apartamento com 3 Quartos à Venda, 83m²",375000.0,83,3,3,2,2693112449,Jardim São Luiz,"(77.0, 86.0]"
9970,Apartamento,"Apartamento com 2 Quartos à Venda, 48m²",175000.0,48,2,1,1,2572455642,Lagoinha,"(45.0, 48.0]"


In [23]:
base_original['metragem_bins'].value_counts()

metragem_bins
(45.0, 48.0]       751
(9.999, 42.0]      683
(58.0, 65.0]       639
(77.0, 86.0]       614
(110.0, 128.0]     604
(96.0, 110.0]      597
(167.0, 4687.0]    595
(128.0, 167.0]     594
(65.0, 70.0]       592
(86.0, 96.0]       590
(52.0, 58.0]       584
(42.0, 45.0]       580
(70.0, 77.0]       530
(48.0, 52.0]       442
Name: count, dtype: int64

In [25]:
fig = px.box(base_original, x='metragem_bins', y='preco', category_orders={})
fig.update_layout( xaxis={
                  'categoryorder': 'category ascending'})

fig.show()

In [26]:
obter_estatistica(base_original=base_original)

Média preço:  450640.89
Mediana Preço: 320000.0
Moda:  250000.0


In [27]:
fig = px.box(base_original, y='preco')
fig.show()

In [28]:
fig = px.histogram(base_original, x='preco', nbins=k)
fig.update_layout(bargap=0.2)
fig.show()

# Teste de hipótese

<ul>
    <li>u1: Média dos imóveis do bairro sumarezinho </li>
    <li>u2: Média dos imóveis do bairro Centro </li>
    <li>H0: u1 <= u2</li>
    <li>H1: u1 > u2 XXX</li>
<ul>

In [58]:
base_original_completa = obter_resumo(tipo_imovel='Apartamento')
base_original_completa

,tipo_imovel,nome,preco,metragem,quarto,banheiro,garagem,ID_CASA,bairro_teste
0,Apartamento,"Apartamento com 2 Quartos à Venda, 65m²",310000.0,65,2,2,1,2699716579,Jardim Botânico
2,Apartamento,"Apartamento com 2 Quartos à Venda, 71m²",403500.0,71,2,2,2,2690493538,Bonfim Paulista
3,Apartamento,"Apartamento com 2 Quartos à Venda, 54m²",290000.0,54,2,2,1,2667652105,Nova Aliança
4,Apartamento,"Apartamento com 2 Quartos à Venda, 45m²",166420.0,45,2,1,1,2682840782,Conjunto Habitacional Jardim Das Palmeiras
5,Apartamento,"Apartamento com 2 Quartos à Venda, 64m²",370000.0,64,2,2,2,2688027260,Vila Ana Maria
...,...,...,...,...,...,...,...,...,...
9967,Apartamento,"Apartamento com 3 Quartos à Venda, 81m²",540000.0,81,3,2,2,2696267182,Vila Do Golf
9968,Apartamento,"Apartamento com Quarto à Venda, 61m²",280000.0,61,1,1,1,2476716299,Centro
9969,Apartamento,"Apartamento com 3 Quartos à Venda, 83m²",375000.0,83,3,3,2,2693112449,Jardim São Luiz
9970,Apartamento,"Apartamento com 2 Quartos à Venda, 48m²",175000.0,48,2,1,1,2572455642,Lagoinha


In [59]:

base_original_completa['bairro_teste'].value_counts().reset_index().query(
    ' bairro_teste in ["Centro","Sumarezinho"] ')

,bairro_teste,count
1,Centro,736
19,Sumarezinho,118


In [60]:
dados_sumarezinho = base_original_completa.query(
    'bairro_teste == "Sumarezinho"').sample(n=50, random_state=42).preco
dados_centro = base_original_completa.query(
    'bairro_teste == "Centro"').sample(n=50, random_state=42).preco

In [61]:
media_amostra_centro = dados_centro.mean()
desvio_padrao_centro = dados_centro.std()

In [62]:
media_amostra_sumarezinho = dados_sumarezinho.mean()
desvio_padrao_sumarezinho = dados_sumarezinho.std()

In [76]:
media_amostra_centro, media_amostra_sumarezinho, media_amostra_centro > media_amostra_sumarezinho

(442434.18, 231241.8, True)

In [63]:
sig = 0.01
conf = 1 - sig
prob = conf
z_alpha = norm.ppf(prob)
z_alpha
D_0 = 0

In [72]:
import numpy as np
numerador = (media_amostra_centro - media_amostra_sumarezinho) - D_0
denominador = np.sqrt((desvio_padrao_centro ** 2 / 100) +
                      (desvio_padrao_sumarezinho ** 2 / 100))

In [73]:
z = numerador / denominador

z

7.665953368080362

In [74]:
z >= z_alpha

True

In [77]:
from statsmodels.stats.weightstats import DescrStatsW, CompareMeans

In [78]:
test_sumarezinho = DescrStatsW(dados_sumarezinho)
test_centro = DescrStatsW(dados_centro)

In [80]:
test_a = test_sumarezinho.get_compare(test_centro)

In [83]:
test_a.ztest_ind(alternative='larger', value=0)

(-5.420647610829478, 0.9999999703082424)

In [84]:
z, p_valor = test_a.ztest_ind(alternative='larger', value=0)

In [85]:
p_valor <= sig

False

In [86]:
test = CompareMeans(test_centro, test_sumarezinho)
test

In [87]:
test_a = CompareMeans(test_centro, test_sumarezinho)

In [88]:
teste_b = CompareMeans(test_centro, test_sumarezinho)